## Data Science Candidate Evaluation: Exploratory Data Analysis for Forecasting Task
### 📄 Introduction
Welcome to the evaluation phase of your application for the Data Science position. This notebook is designed to assess your skills in exploratory data analysis (EDA) and forecasting. You will be provided with a dataset and a series of tasks aimed at testing your ability to interpret data and extract meaningful insights that could inform future forecasting models.

### 🎯 Objective
Your main objective in this notebook is to conduct a thorough exploratory analysis of the provided dataset. You should aim to understand the underlying patterns, trends, and anomalies within the data. The insights you generate here will be pivotal for building robust forecasting models in later stages.

### 📚 Dataset Description
* 	📆 Calendar.csv
    - DATE
    - STORE_ID
    - STATE
        - C : Closed
        - A : Open
        - FA : Open on holiday
        - FM : Open on holiday until noon
        - CR : Closed for renovation
    - OPENING_TIME
    - CLOSING_TIME 
    - HOLIDAY_AND_CLOSED
        - 1: True,  0: False
    - UNDER_REFORM
        - 1: True, 0: False
* 📉 ipc_history.csv
    - Evolution of IPC over time
* 📝 sales_test_submission.csv
    - File for results. Colums SALES must be filled with predictions
* 📈 sales_train_dataset.csv
    - Cumulative sales over time



##### Task 1: Import all necessary libraries

In [ ]:
...

##### Task 2: Load all CSV files (format data if necessary)

In [ ]:
...

##### Task 3: Basic sales exploration
* Plot the sales over the year
* Bar graph with sales grouped by month
* Which day of the week (on average) does Mercadona have the best sales?

In [ ]:
...

##### Task 4: Outliers
* Identify possible outliers
* ¿Are there missing values?, ¿How would you deal with them?

In [ ]:
...

##### Taskk 5: Open/Close datase
* Plot a two-line graph. Each line should represent the amount of stores open/close at each day

In [ ]:
...

##### Task 6: Seasonality
* ¿Does sales follow a trend?
* ¿Can you identify a pattern on sales?
* ¿Are sales regular over all the year?

In [13]:
...

Ellipsis

##### Task 7: Optionaly
* If consider there are any kind of analysis that still can be done and would be useful for a forecasting task, that's the place to do it

In [ ]:
...

##### Task 8: Dataset creation
* At this stage, you should be capable of creating a dataset for a forecasting task. The goal is to populate the SALES column in the test submission file. To achieve this, you will need to first create a dataset and then train a model. You are encouraged to utilize all available information in the CSV files (and optionally, from the internet) to compile a single CSV file containing all chosen features along with the target variable.
* tip: Depending on the model you want to use, it's possible that categorical variables needs a special processing 😉.